In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt 
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from skimage.transform import resize
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from sklearn.preprocessing import MinMaxScaler
import os

In [ ]:
# data from https://www.floydhub.com/emilwallner/datasets/colornet
# download data an put it in a folder in the root called "data"
items = []
for file in os.listdir("./data/images/Train/"):
    img_array = img_to_array(load_img("./data/images/Train/" + file))
    items.append(img_array)
items = np.array(items)
X_train = 1.0/255 * items



In [ ]:
# # splits the data into train and test sets
# X_train, X_test, Y_train, Y_test = train_test_split(imgs_gray, imgs_ab, test_size=0.2, random_state=42, shuffle=True)
# X_train.astype('float')
# X_test.astype('float')
# Y_train.astype('float')
# Y_test.astype('float')
# print("X_train " + str(X_train.shape))
# print("X_test " + str(X_test.shape))
# print("y_train " + str(Y_train.shape))
# print("y_test " + str(Y_test.shape))

# print(X_train.min())
# print(X_train.max())

# print(Y_train.min())
# print(Y_train.max())



In [ ]:
# loads the inceptionResNetV2 for prediction of large features of the images
import tensorflow.compat.v1 as tf_compact
tf_compact.disable_v2_behavior()
inception = InceptionResNetV2(weights=None, include_top=True)
inception.load_weights("./data/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels.h5")
inception.graph = tf_compact.get_default_graph()

In [ ]:
# making the prediction with inceptionResNet
# them the prediction is embed into the imgs
def embed_inception_prediction(img_unsized):
    # resize the image to fit in the resNet
    img_resized = []
    for i in img_unsized:
        img = resize(i, (299, 299, 3), mode='constant')
        img_resized.append(img)
    img_resized = np.array(img_resized)
    # this prepares the data for the resNet
    img_resized = preprocess_input(img_resized)
    with inception.graph.as_default():
        imgs_embed = inception.predict(img_resized)
    return imgs_embed
    


In [ ]:
train_datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True
)

batch_size_number = 10 # batch

def image_train_datagen(X_data, batch_size):
    for batch_imgs in train_datagen.flow(X_data, batch_size=batch_size):

        grayscale_for_embeding = gray2rgb(rgb2gray(batch_imgs))
        embeding = embed_inception_prediction(grayscale_for_embeding)

        lab_train = rgb2lab(batch_imgs)
        l_batch = lab_train[:, :, :, 0]
        l_batch = l_batch.reshape(l_batch.shape + (1,))

        ab_batch = lab_train[:, :, :, 1:] / 128
       
        yield([l_batch, embeding], ab_batch)

                

In [ ]:
# def test_image_train_datagen(X_data, batch_size):
#     for batch_imgs in train_datagen.flow(X_data, batch_size=batch_size):
#         grayscale_for_embeding = gray2rgb(rgb2gray(batch_imgs))
#         embeding = embed_inception_prediction(grayscale_for_embeding)

#         lab_train = rgb2lab(batch_imgs)
#         l_batch = lab_train[:, :, :, 0]
#         l_batch = l_batch.reshape(l_batch.shape + (1,))
#         print(l_batch.shape)

#         ab_batch = lab_train[:, :, :, 1:] / 128
#         print(l_batch.shape)
#         print(l_batch[0])

#         print(ab_batch.shape)
#         print(ab_batch[0])
#         return([l_batch, embeding], ab_batch)


# data = np.array([X_train[0]])
# print()
# test = test_image_train_datagen(data, 1)


In [31]:
from tensorflow.keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, concatenate, Activation, Dense, Dropout, Flatten, RepeatVector
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers.core import , Permute


ModuleNotFoundError: No module named 'tensorflow.keras.layers.core'

In [ ]:

test_img = cv2.imread("./data/testImg.jpeg")
rgb_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
print (rgb_img.shape)

display(rgb_img)
rgb_img.astype("uint8")
img_lab = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2LAB)
img_lab = rgb2lab(test_img)
img_l = img_lab[:, :, 0]
img_ab = img_lab[:, :, 1:]
print(img_l.max())
print(img_l.min())
print(img_ab.max())
print(img_ab.min())
display(img_l)

In [ ]:


x_batch = []
y_batch = []

x_batch.append(X_train[3])
y_batch.append(Y_train[3])

x_batch = np.array(x_batch)
y_batch = np.array(y_batch)
display(X_train[0])


batch = 1


def image_l_ab_datagen(x_train, y_train, batch_size):
    for x_batch, y_batch in training_data_gen.flow(x_train, y_train, batch_size=batch_size):
        x_batch = np.array(x_batch)
        y_batch = np.array(y_batch)

        gray_embeding = embed_inception_prediction(x_batch)
        # to embeding here
        
        x_train = (x_batch * (100/255)) / 100
        y_train = y_batch / 128
        yield([x_train, gray_embeding], y_train)

# imgs_rgb_normalized = []
# for i in range(len(x_batch)):
#     rgb_normalized = lab2RGB(x_batch[i], y_batch[i])
#     # rgb = rgb * 255
#     # rgb_normalized = 1.0/255 * rgb_normalized
#     imgs_rgb_normalized.append(rgb_normalized)
# imgs_rgb_normalized = np.array(imgs_rgb_normalized)
# display(imgs_rgb_normalized[0])
# gray_embeding = embed_inception_prediction(x_batch)
# # to embeding here=
# imgs_lab_normalized = rgb2lab(255 *imgs_rgb_normalized)
# x_train = imgs_lab_normalized[:, :, :, 0]
# x_train = x_train.reshape(x_train.shape + (1,))

# y_train = imgs_lab_normalized[:, :, :, 1:]

# y_train[0] =  y_train[0]
# display(x_train[0])
# img_display = lab2RGB(x_train[0], y_train[0])
# display(img_display)

# print(([x_train, gray_embeding], y_train))

print(x_batch.shape)


# embeding predictions
gray_embeding = embed_inception_prediction(x_batch)

imgs_lab = np.zeros((x_batch.shape[0], x_batch.shape[1], x_batch.shape[2], 3))
print(imgs_lab.shape)
imgs_lab[:, :, :, 0] = x_batch[:, :, :, 0]
imgs_lab[:, :, :, 1:] = y_batch

img_lab = imgs_lab[0]
img_lab = img_lab.astype('uint8') 
print(imgs_lab[0].shape)
img_rgb = cv2.cvtColor(img_lab, cv2.COLOR_LAB2RGB)

imgs_lab_norm = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2LAB)

# imgs_l = imgs_lab_norm[:, :, 1:] / 128

imgs_lab_norm_l = imgs_lab_norm[:, :, 0]
imgs_lab_norm_ab = imgs_lab_norm[:, :, 1:]
print(imgs_lab_norm_l.max())
print(imgs_lab_norm_l.min())
print(imgs_lab_norm_ab.max())

print(imgs_lab_norm_ab.min())
# print(imgs_l.max())
# print(imgs_lab)


x_train = (x_batch * (100/255)) / 100

y_train = y_batch / 128
print(y_train.max())
print(y_train.min())
display(x_train[0])
y_train = y_train * 128

x_train = (x_train * 100) * (255/100)

img_to_display = lab2RGB(x_train[0], y_train[0])
display(img_to_display)
# yield([x_train, gray_embeding], y_train)

In [ ]:
def get_rbg_from_lab(gray_imgs, ab_imgs, n = 10):
    
    #create an empty array to store images
    imgs = np.zeros((n, 224, 224, 3))
    
    imgs[:, :, :, 0] = gray_imgs[0:n:]
    imgs[:, :, :, 1:] = ab_imgs[0:n:]
    
    #convert all the images to type unit8
    imgs = imgs.astype("uint8")
    
    #create a new empty array
    imgs_ = []
    
    for i in range(0, n):
        imgs_.append(cv2.cvtColor(imgs[i], cv2.COLOR_LAB2RGB))

    #convert the image matrix into a numpy array
    imgs_ = np.array(imgs_)

    #print(imgs_.shape)
    
    return imgs_

In [ ]:
def lab2RGB(l, ab):
    shape = (l.shape[0],l.shape[1],3)
    img = np.zeros(shape)
    img[:,:,0] = l[:,:,0]
    img[:,:,1:] = ab
    img = img.astype('uint8')
    print(img.max())
    img = cv2.cvtColor(img, cv2.COLOR_LAB2RGB)
    return img
def display(img):
    plt.figure()
    plt.set_cmap('gray')
    plt.imshow(img)
    plt.show()


img = lab2RGB(X_train[0], Y_train[0])
display(img)
    
# testing_resacel = testing_resacel * 255
# testing_res_a_b = testing_res_a_b * 255

# img = l_ab_to_RGB(X_train[1], y_train[1])
# # img = l_ab_to_RGB(testing_resacel, testing_res_a_b)
# # img = np.resize(img, (256, 256, 3))
# img = np.array(resize(img, (300, 300, 3), mode='constant'))
# print(img.shape)

# # testing_resacel = testing_resacel * 255
# # display(testing_resacel)
# display(img)

In [ ]:
# for loading and image to predict

test_img = cv2.imread("./data/ww1.jpeg")
rgb_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
print (rgb_img.shape)

display(rgb_img)

img_lab = rgb2lab(test_img)
img_l = img_lab[:, :, 0]
img_ab = img_lab[:, :, 1:]
print(img_l.max())
print(img_l.min())
print(img_ab.max())
print(img_ab.min())
display(img_l)